<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 3.2.1
# *Querying the International Space Station*

## The OpenNotify API

The OpenNotify API exposes a few attributes of the International Space Station (ISS) via a simple, authentication-free interface. The simplicity of this API precludes any need for a dedicated Python library. However, as with many APIs, it accepts requests according to HTTP standards and returns responses in JSON format, so the Python libraries request and json will make managing the I/O simpler still.

In [2]:
import requests
import json
from datetime import datetime, date, time

This request fetches the latest position of the international space station:

In [3]:
response = requests.get("http://api.open-notify.org/iss-now.json")

Print the status code and text of the response:

In [4]:
#ANSWER
response.status_code

200

In [5]:
#ANSWER
print(response.text)

{"message": "success", "iss_position": {"longitude": "1.4436", "latitude": "-9.9736"}, "timestamp": 1722463263}


We can use another API to request the current position of the ISS and the next few times at which it will be over a certain location. The latitude and longitude of Sydney are (-33.87, 151.21).

In [6]:
response = requests.get("https://api.g7vrd.co.uk/v1/satellite-passes/25544/-33.87/151.21.json?minelevation=0&hours=24")

Print the response header:

In [7]:
#ANSWER
print(response.headers)

{'Date': 'Wed, 31 Jul 2024 22:01:06 GMT', 'Server': 'Apache', 'Vary': 'Origin,Access-Control-Request-Method,Access-Control-Request-Headers', 'Access-Control-Allow-Origin': '*', 'X-Content-Type-Options': 'nosniff', 'X-XSS-Protection': '0', 'Cache-Control': 'no-cache, no-store, max-age=0, must-revalidate', 'Pragma': 'no-cache', 'Expires': '0', 'X-Frame-Options': 'DENY', 'Content-Type': 'application/json', 'Keep-Alive': 'timeout=5, max=100', 'Connection': 'Keep-Alive', 'Transfer-Encoding': 'chunked'}


Print the content of the response (the data that the server returned):

In [8]:
#ANSWER
print(response.content)

b'{"api_status":"ALPHA","request_timestamp":"2024-07-31T22:01:06.279528238Z","norad_id":25544,"satellite_name":"ISS","tle_last_retrieved":"2024-07-30T22:42:21.328009406Z","lat":-33.87,"lon":151.21,"hours":24,"min_elevation":0,"query_ms":16,"passes":[{"start":"2024-07-31T22:30:31.263Z","tca":"2024-07-31T22:35:01.263Z","end":"2024-07-31T22:39:51.263Z","aos_azimuth":248,"los_azimuth":4,"max_elevation":13.0},{"start":"2024-08-01T11:57:31.263Z","tca":"2024-08-01T12:01:31.263Z","end":"2024-08-01T12:06:06.263Z","aos_azimuth":2,"los_azimuth":107,"max_elevation":10.0},{"start":"2024-08-01T13:32:41.263Z","tca":"2024-08-01T13:38:11.263Z","end":"2024-08-01T13:43:36.263Z","aos_azimuth":306,"los_azimuth":136,"max_elevation":58.0},{"start":"2024-08-01T15:11:01.263Z","tca":"2024-08-01T15:15:31.263Z","end":"2024-08-01T15:19:51.263Z","aos_azimuth":257,"los_azimuth":150,"max_elevation":10.0},{"start":"2024-08-01T16:50:46.263Z","tca":"2024-08-01T16:53:46.263Z","end":"2024-08-01T16:56:36.263Z","aos_azimuth

Note that this is a Python byte string:

In [9]:
print(type(response.content))

<class 'bytes'>


Print just the "content-type" value from the header:

In [10]:
#ANSWER
print(response.headers['content-type'])

application/json


JSON was designed to be easy for computers to read, not for people. The `requests` library can decode the JSON byte string:

In [11]:
overheads = response.json()
print(overheads)

{'api_status': 'ALPHA', 'request_timestamp': '2024-07-31T22:01:06.279528238Z', 'norad_id': 25544, 'satellite_name': 'ISS', 'tle_last_retrieved': '2024-07-30T22:42:21.328009406Z', 'lat': -33.87, 'lon': 151.21, 'hours': 24, 'min_elevation': 0, 'query_ms': 16, 'passes': [{'start': '2024-07-31T22:30:31.263Z', 'tca': '2024-07-31T22:35:01.263Z', 'end': '2024-07-31T22:39:51.263Z', 'aos_azimuth': 248, 'los_azimuth': 4, 'max_elevation': 13.0}, {'start': '2024-08-01T11:57:31.263Z', 'tca': '2024-08-01T12:01:31.263Z', 'end': '2024-08-01T12:06:06.263Z', 'aos_azimuth': 2, 'los_azimuth': 107, 'max_elevation': 10.0}, {'start': '2024-08-01T13:32:41.263Z', 'tca': '2024-08-01T13:38:11.263Z', 'end': '2024-08-01T13:43:36.263Z', 'aos_azimuth': 306, 'los_azimuth': 136, 'max_elevation': 58.0}, {'start': '2024-08-01T15:11:01.263Z', 'tca': '2024-08-01T15:15:31.263Z', 'end': '2024-08-01T15:19:51.263Z', 'aos_azimuth': 257, 'los_azimuth': 150, 'max_elevation': 10.0}, {'start': '2024-08-01T16:50:46.263Z', 'tca': '2

What kind of object did this give us?

In [12]:
#ANSWER:
print(type(overheads))

<class 'dict'>


Python dicts are easier to work with, but the data we want is still buried in that data structure, so we have to dig it out. First, extract the `passes` value to a separate list:

In [13]:
#ANSWER:
passes = overheads['passes']

In [14]:
passes

[{'start': '2024-07-31T22:30:31.263Z',
  'tca': '2024-07-31T22:35:01.263Z',
  'end': '2024-07-31T22:39:51.263Z',
  'aos_azimuth': 248,
  'los_azimuth': 4,
  'max_elevation': 13.0},
 {'start': '2024-08-01T11:57:31.263Z',
  'tca': '2024-08-01T12:01:31.263Z',
  'end': '2024-08-01T12:06:06.263Z',
  'aos_azimuth': 2,
  'los_azimuth': 107,
  'max_elevation': 10.0},
 {'start': '2024-08-01T13:32:41.263Z',
  'tca': '2024-08-01T13:38:11.263Z',
  'end': '2024-08-01T13:43:36.263Z',
  'aos_azimuth': 306,
  'los_azimuth': 136,
  'max_elevation': 58.0},
 {'start': '2024-08-01T15:11:01.263Z',
  'tca': '2024-08-01T15:15:31.263Z',
  'end': '2024-08-01T15:19:51.263Z',
  'aos_azimuth': 257,
  'los_azimuth': 150,
  'max_elevation': 10.0},
 {'start': '2024-08-01T16:50:46.263Z',
  'tca': '2024-08-01T16:53:46.263Z',
  'end': '2024-08-01T16:56:36.263Z',
  'aos_azimuth': 215,
  'los_azimuth': 151,
  'max_elevation': 3.0},
 {'start': '2024-08-01T18:28:16.263Z',
  'tca': '2024-08-01T18:32:16.263Z',
  'end': '2024

Now extract the `start` strings into an array called `srisetimes`:

In [15]:
#ANSWER:
srisetimes = [xpass['start'] for xpass in passes]

In [16]:
srisetimes

['2024-07-31T22:30:31.263Z',
 '2024-08-01T11:57:31.263Z',
 '2024-08-01T13:32:41.263Z',
 '2024-08-01T15:11:01.263Z',
 '2024-08-01T16:50:46.263Z',
 '2024-08-01T18:28:16.263Z',
 '2024-08-01T20:04:31.263Z',
 '2024-08-01T21:41:16.263Z',
 '2024-08-02T11:10:01.263Z']

These are strings. We convert these to an array of Python `datetime` values called `risetimes`:

In [17]:
risetimes = [datetime.strptime(xpass['start'], "%Y-%m-%dT%H:%M:%S.%fZ") for xpass in passes]
risetimes

[datetime.datetime(2024, 7, 31, 22, 30, 31, 263000),
 datetime.datetime(2024, 8, 1, 11, 57, 31, 263000),
 datetime.datetime(2024, 8, 1, 13, 32, 41, 263000),
 datetime.datetime(2024, 8, 1, 15, 11, 1, 263000),
 datetime.datetime(2024, 8, 1, 16, 50, 46, 263000),
 datetime.datetime(2024, 8, 1, 18, 28, 16, 263000),
 datetime.datetime(2024, 8, 1, 20, 4, 31, 263000),
 datetime.datetime(2024, 8, 1, 21, 41, 16, 263000),
 datetime.datetime(2024, 8, 2, 11, 10, 1, 263000)]

Finally, use `risetime.strftime` to print these in a format that people understand:

```
e.g.
18/10/22 07:05
18/10/22 08:41
18/10/22 10:20
18/10/22 12:00
18/10/22 01:37
18/10/22 03:13
```



In [18]:
#ANSWER:
for risetime in risetimes:
    print(risetime.strftime('%d%m%y%I:%M'))

31072410:30
01082411:57
01082401:32
01082403:11
01082404:50
01082406:28
01082408:04
01082409:41
02082411:10


Finally, here is an endpoint that tells us who is on board:

In [19]:
response = requests.get("http://api.open-notify.org/astros.json")

In [20]:
response

<Response [200]>

In [21]:
print(response.text)

{"people": [{"craft": "ISS", "name": "Oleg Kononenko"}, {"craft": "ISS", "name": "Nikolai Chub"}, {"craft": "ISS", "name": "Tracy Caldwell Dyson"}, {"craft": "ISS", "name": "Matthew Dominick"}, {"craft": "ISS", "name": "Michael Barratt"}, {"craft": "ISS", "name": "Jeanette Epps"}, {"craft": "ISS", "name": "Alexander Grebenkin"}, {"craft": "ISS", "name": "Butch Wilmore"}, {"craft": "ISS", "name": "Sunita Williams"}, {"craft": "Tiangong", "name": "Li Guangsu"}, {"craft": "Tiangong", "name": "Li Cong"}, {"craft": "Tiangong", "name": "Ye Guangfu"}], "number": 12, "message": "success"}


Referring to the methods used above, extract the number of astronauts and their names:

In [22]:
#ANSWER:
astros = response.json()
print(astros)
print(astros['number'])
for astronaut in astros['people']:
    print(astronaut['name'])

{'people': [{'craft': 'ISS', 'name': 'Oleg Kononenko'}, {'craft': 'ISS', 'name': 'Nikolai Chub'}, {'craft': 'ISS', 'name': 'Tracy Caldwell Dyson'}, {'craft': 'ISS', 'name': 'Matthew Dominick'}, {'craft': 'ISS', 'name': 'Michael Barratt'}, {'craft': 'ISS', 'name': 'Jeanette Epps'}, {'craft': 'ISS', 'name': 'Alexander Grebenkin'}, {'craft': 'ISS', 'name': 'Butch Wilmore'}, {'craft': 'ISS', 'name': 'Sunita Williams'}, {'craft': 'Tiangong', 'name': 'Li Guangsu'}, {'craft': 'Tiangong', 'name': 'Li Cong'}, {'craft': 'Tiangong', 'name': 'Ye Guangfu'}], 'number': 12, 'message': 'success'}
12
Oleg Kononenko
Nikolai Chub
Tracy Caldwell Dyson
Matthew Dominick
Michael Barratt
Jeanette Epps
Alexander Grebenkin
Butch Wilmore
Sunita Williams
Li Guangsu
Li Cong
Ye Guangfu


## HOMEWORK


1. Write a simple handler for the response status code (refer to lab resources slide for HTTP response codes). As this Jupyter Notebook is an interactive device, the handler does not need to manage subsequent code execution (i.e. by branching or aborting execution), although it should return something that could be used to do so if deployed in a Python program.

In [23]:
#ANSWER:
def handleResponse(response, verbose = False):
    '''
    Returns Boolean Value, Status Code,
    '''
  # if Status Code is 200 return false, and status code
  # Otherwise Return True and Status Code

2. Test your response handler on some correct and incorrect API calls.

In [24]:
response = requests.get("http://api.open-notify.org/astros.json")
if handleResponse(response):
    print('API call failed. Resolve issue before continuing!')

response = requests.get("http://api.open-notify.org/iss-now.json")
handleResponse(response, True)

>

>

>



---



---



> > > > > > > > > © 2024 Institute of Data


---



---



